<a href="https://colab.research.google.com/github/Johyeyoung/capstone-2021-35/blob/master/Transaction/Data/action_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 단어 리스트만들기

## 데이터 가져오기

In [6]:
import pandas as pd
import re

df = pd.read_csv('/content/drive/MyDrive/2-2._lesion_info_table.csv', encoding='cp949')

In [7]:
# 같은 내용의 카테고리 합치기
df["lesion_detail_type"] = df["lesion_detail_type"].str.replace(pat=r'하이패스_분실', repl=r'하이패스_카드분실', regex=True)

# 3차 level의 종류
inspect_type=df['lesion_detail_type']

inspect_type=list(set(inspect_type.values.tolist()))
print(sorted(inspect_type))
print(len(inspect_type))

['ADAS_고장', 'ADAS_파손', 'USB포트_고장', '거치대_분실', '거치대_파손', '경고등_ABS', '경고등_DPF(배출가스)', '경고등_ESC(차세제어)', '경고등_배터리', '경고등_브레이크', '경고등_스패너(서비스알림)', '경고등_에어백', '경고등_엔진오일', '경고등_엔진체크', '경고등_연료부족', '경고등_온도게이지', '경고등_요소수부족', '경고등_전/후측방 충돌 경고', '경고등_통합경고등(느낌표)', '글로브박스_파손', '내비게이션_고장', '내비게이션_분실', '내비게이션_업데이트', '내비게이션_인터넷연결불가', '내비게이션_파손', '내장제_파손', '단말기_배선', '단말기_신호불량', '단말기_탈거', '단말기_파손', '도어_고장', '도어_소음', '도어_파손', '라이트/전구_경고등(고장)', '라이트/전구_파손', '룸미러_파손', '목받침대_고장/분실', '문제어불가_옥스트라', '문제어불가_카드태깅', '발판매트_분실', '번호판_분실', '번호판_파손', '본넷_고장', '본넷_파손', '브레이크_고장', '브레이크_밀림', '브레이크_소음', '블랙박스_고장', '블랙박스_분실', '블랙박스_탈거', '블랙박스_파손', '사고조사_요청', '사이드미러_고장', '사이드미러_파손', '사이드브레이크_해제불가', '선바이저_파손', '시거잭_고장', '시동_고장', '시동_꺼짐', '시동_배터리방전', '시동_키인식불가(키아웃)', '시동_혼유', '시트_시트조절_고장', '시트_열선시트_고장', '시트_파손', '실내등_고장', '안심번호_분실', '안심번호_연동오류', '안전벨트_고장', '에어컨_고장', '에어컨_소음', '에어컨_악취', '오디오_AUX', '오디오_고장', '오디오_버튼파손/분실', '오디오_블루투스', '오디오_소음', '와이퍼_고장', '와이퍼_교체필요', '와이퍼_파손', '외관_누유', '외관_파손(상부)', '외관_파손(하부)', '외부스티커_훼손', '워셔액_

### 키워드 추출기

### 키워드추출기

In [119]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip install customized_konlpy # 형태소 사용자 사전 추가
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기
!pip install krwordrank # 키워드 추출기
!pip install kss # 문장분리 

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [23]:
!pip3 install krwordrank
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize


def keyword_tracker_KR(trigger):
  texts = [normalize(text, english=True, number=True) for text in trigger]  # 문장내 특수문자 제거하기
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 10, # 단어의 최대 길이
      verbose = True
      )
  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10
  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
  keyword = []
  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
      print('%8s:\t%.4f' % (word, r))
      keyword.append(word)
  return keyword

#형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt
okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls

def keyword_tracker_Okt(memo): # memo-문장이 들어있는 리스트
  memo = [ re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',text) for text in memo]  # 문장내 특수문자 제거하기
  memo = ' '.join(memo)
  memo=morph_and_stopword(memo) # 형태소 분리
  memo=Counter(memo)
  memo_rank=memo.most_common(5) # 상위 5개
  return memo_rank



''' 
print(' \n▶ 「Okt」형태소 분석:')
keyword_tracker_Okt(token)

print(' \n▶ 「KRWordRank」형태소 분석:')
keyword_tracker_KR(token)
'''


" \nprint(' \n▶ 「Okt」형태소 분석:')\nkeyword_tracker_Okt(token)\n\nprint(' \n▶ 「KRWordRank」형태소 분석:')\nkeyword_tracker_KR(token)\n"

In [24]:
# 부산대 맞춤법
'''
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium


import sys 
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') 
from selenium import webdriver 
import time 

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless') 
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dev-shm-usage') 
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options) 

def spell_checking(sentence): 
  time.sleep(0.5) 
  wd.get('http://speller.cs.pusan.ac.kr/') 
  try: 
    wd.find_element_by_xpath('//*[@id="text1"]').send_keys(sentence) 
    wd.find_element_by_xpath('//*[@id="btnCheck"]').click() 
    time.sleep(1) 
    entity_num = 0 
    while True: 
      try: 
        wd.find_element_by_xpath('//*[@id="tdReplaceWord_'+str(entity_num)+'"]/ul/li/a').click() 
        entity_num += 1 
      except:
        break 
    texts = wd.find_element_by_xpath('//*[@id="tdCorrection1stBox"]').text 
    return texts 
  except: 
    return sentence
'''



Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

ModuleNotFoundError: ignored

In [84]:
# 형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt
okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls


# 맞춤법, 띄어쓰기 교정 - 텍스트 전처리
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

### 각 원인별 데이터

In [26]:
def customer_info():
    phone_number='(인입번호|[연락]*[\s]*가능[\s]*한[\s]*번호)[^\n]*'
    url='(관제\s예약\s)*URL[주소:\s]*[^\n]+'
    location='(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*'
    lst = [phone_number, url, location]
    return '|'.join(lst)

def service_info():
    case_number='([원]*예약번호|블락번호)[^\n]*'
    #card='(사고카드|장애카드|장애블락)[\s][^\n]*' # 장애카드 생성, 장애블락 생성
    #picture='(사진[\s]*확인|사진[\s]*여부)[^\n]*' # 사진 여부 | 확인 
    customer='((고객[\s]*과실[\s]*(사유|여부))|(현장[\s]*고객[\s]*유무)|([^\n]*본인[\s]*확인)|(고객운행중))[^\n]*'
    chatting='([\\n]*(채팅[\s]*상담|톡|요청자)[\s]*[^\n]*)|([\\n]*선제[(조치)(장애)(건)\s]*)|CC'  # 채팅, 톡 /선제조치건/ 사진여부 /본인확인/요청자 제거
    #lst = [case_number, card, picture, customer, chatting]
    lst = [case_number, customer, chatting]
    return '|'.join(lst)



In [27]:
import re

factor_trgr = re.compile(r'문의[\s]*내용|증상[\s]*상세|문의[\s]*사항')
factor_check = re.compile(r'확인[\s]*[내용|처리[\s]*내용]*|확인[\s]*사항') 
factor_figure = re.compile(r'안내[\s]*[약속[\s]*내용|처리[\s]*사항|처리[\s]*내용|사항]|안내사항') 
factor_promise = re.compile(r'약속[\s]*[내용|사항]') 


def track_Trigger(memo):
  # 1....... 주어진 문장을 \n로 분할하여 리스트만들기
  memo = re.sub(r'( )*(\n+)( )*', '\n', memo)
  memo = re.sub(r'(\n+)', '\n', memo)
  memo_chunck = memo.split('\n')

  # 2............   필요한 리스트 초기화
  flag = 'trigger_mode'
  figure=[]
  check=[]
  trigger=[]
  money=[]

  # 3.................... 쪼개진 문장에서 Trigger / 확인사항/ figure / 나누기 
  for i in memo_chunck:
      if factor_check.search(i): 
        flag = 'check_mode'
        if len(i.split(':')[1:]) > 1: check.append(str(i.split(':')[1:])) 
        else: continue
      elif factor_figure.search(i): 
        flag = 'figure_mode'
        if len(i.split(':')[1:]) > 1: figure.append(str(i.split(':')[1:])) 
        else: continue
      elif factor_promise.search(i): 
        flag = 'promise_mode'   

      if flag == 'trigger_mode':  trigger.append(" ".join(i.split(':')[1:])) if len(i.split(':')) > 1 else trigger.append(i)
      elif flag == 'check_mode':  check.append(i)
      elif flag == 'figure_mode': figure.append(i) 
      elif flag == 'promise_mode':  pass

  # 4........ 결과 출력하기
  print('▷ Chunk: ', memo_chunck)
  if len(trigger) != 0: print("증상상세:", trigger)
  if len(check)  != 0: print("확인내용:", check)
  if len(figure) != 0: print("안내처리:", figure)
  if len(figure) != 0: print("안내처리:", refine_figure(figure))

  return figure, check, trigger 

In [206]:
from numpy import nan

# action부분에서 이러한 단어가 있으면 중요한 문장으로 생각한다
#스피드|메이트|TS|ts|티스테이션|현대|가온|입고|업체|점$
actor_figure =  '마감|교환|교체|테이프|엔,()*와,()*향|강제|개방|진단|중계기|보충|조절|OTP|삽입|테스트|바디|청소|컴파운드|장착|워시|불량|퓨즈|업그[래|레]이드|유심|'
actor_figure += '안내[\s]*[약속[\s]*내용|처리[\s]*사항|처리[\s]*내용|사항]|안내사항|TCS|티씨에스|네비게이션|음성|부팅|오토|충전소|원격|부착|점프|스캐[너|서]|배치|라우터|등받이|나사|주유|'
actor_figure += '누유|시거잭|배출|가스|가리개|글로브박스|업데이트|내장제|공기압|배선|후방|센서|ESC|탈거|환경이슈|전좌|버튼|전우|발판|매트|포켓|만차|인터넷|목받침대|블박|블랙[\s]*박스|시트조절|하부|안심번호|관리인요청|파손|얼라이먼트|신호|타는냄새|위치조절|AUX|출차불가|배터리|휠|'
actor_figure += '블루투스|스패너|혼유|알림|펑크|상부|엑셀|급발진|고장|[선|썬]루프|썬|[선|썬]바이저|시동|USB|포트|히터|타이어|꺼|들리|얼어|언|녹이|따뜻|차가운|떨어|찬|깜[빡|박]|알림|쏠|찢|뽑|꼽|닫|충돌|방지|보조|에이필러|바람[\s]*빠|'
actor_figure += '태깅|연결|외부스티커|에어|RPM|계기판|가스|요소수|위생불량|DPF|GPS|스마트[\s]*키|범퍼|SD|카드|경보|소음|마스터키|소리|악취|구토|ABS|고장|손상|차량[\s]*떨림|떨리|ADAS|번호판|오류|유리|오일|외관|부족|카메라|냄새|담뱃재|온도게이지|변속기|이상|보관함|코드절상|'
actor_figure += '신호|탐색|기름|방전|충전|연료|견인|인식|미러|사이드|손잡이|글로브[\s]*박스|바퀴|마모|공업사|하이패스|불[\s]*빛|라이트|캡|분실|볼륨|고정|네트[\s]*워크|공조|장치|유리|습기|수동|제어|잠금|열|닫|전조등|실내등|경고등|주행등|제동등|점등|경고|후미등|시야[\s]*[확보]*|조수석|[자|좌]석|재시작|[하|상]향|각도|컨트롤|패드|스크래치|단차|내부|엔진|운전석|보험|동파|모드|방향|판단|편의|품목|접촉|훼손|통신|초기화|전원|안나|내비|사고조사|실행|화면|와이파이|창문|증상|후진|기어|오디오|트렁크|단말기|장착물|핸들떨림|후좌|열선|후측방 충돌|밀림|해제불가|본넷|후우|차세제어|에어컨|중계기점검|브레이크|배터리방전|와이퍼|옥스트라|변속레버|외부위생불량 |카드오류|도어|체크|연동|파스|교체필요|입차불가|전구|커넥터|안전벨트|키아웃|핸들|열선시트|비치함|거치대|담배|시트|위생|[와|워]셔액| 차량위치확인불가|썬팅|경적|키인식|꺼짐|동물털|충전[\s]*카드|주유[\s]*카드'
actor_figure = re.compile(actor_figure) 


def refine_action(action_list):
  print('before:', action_list)
  action_list = [x for x in action_list if x is not nan]
  action_list = "\n".join(action_list)
  action_list = re.sub(r'ㄴ|ㅁ|#|_|(\t)', ' ', action_list)
  action_list = re.sub(r'\([0-9ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z/(\-),: ]*\)|([0-9]+/[0-9]+)', '', action_list) # 삭제: 괄호 내용 & 날짜 
  action_list = re.sub(r'[^\n]*[ㅡ|\-|:|\|]', '\n', action_list)
  action_list = re.sub(r'[네|내]비[게이션]*', '내비게이션', action_list)
  action_list = re.sub(r'재삽입', '재장착', action_list)
  action_list = re.sub(r'교환', '교체', action_list)
  action_list = re.sub(r'블박', '블랙박스', action_list)
  action_list = re.sub(r'(좌[ /.,]*우)|(전[ /.,]후)', '', action_list)
  action_list = re.sub(r'[전|후]+[ ]*[좌|우]+', '', action_list)
  action_list = re.sub(r'[전|후|앞|뒤]+( )*패드', '패드', action_list)
  action_list = re.sub(r'[전|후|앞|뒤]+( )*디스크', '디스크', action_list)
  action_list = re.sub(r'[전|후|앞|뒤]+( )*타이어(2( )*본)*', '타이어', action_list)
  action_list = re.sub(r'[전|후|앞|뒤]+( )*드럼', '드럼', action_list)
  action_list = re.sub(r'( )*(\n+)( )*', '\n', action_list)
  action_list = re.sub(r'(\n+)', '\n', action_list)
  #print("before", [action_list])
  action_list = re.split('\n|/|\-|ㅡ|\.', action_list.upper()) # 문장분리
  
 
  # import kss 
  # for i in action_list[:]:
  #   split_sent = kss.split_sentences(i)
  #   if len(split_sent) > 1:
  #     action_list.remove(i)
  #     action_list += split_sent
  real_action_list = []
  no_use = re.compile(r'->|[0-9]{3,10}|@|가온|스피드|담당자|이상( )*없') 
  for i in action_list[:]:
    i = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',i)
    i = correction(i.lstrip()) # 맞춤법 교정하기 2 (네이버 검사기)
    if actor_figure.search(i):
      if not no_use.search(i): #완벽하게 통과된 
        real_action_list.append(i)
  return real_action_list

In [207]:
# csv 파일로 저장하기 


# f_C = open('/content/drive/MyDrive/action_list.txt','w')
# f_C.write('\n{} ({}개): '.format(inspect_type[idx],  len(b)))
# f_C.write(str(sorted(b)))
 




import re

dic_trigger={}
dic_check={}
dic_figure={}
dic_action={}

inspect_num=0
# lesion_detail_type 그룹별 데이터 확인
#inspect_type = ['하이패스_고장', '하이패스_카드분실', '하이패스_카드오류', '하이패스_파손(탈거, 파손)']
#inspect_type += ['내비게이션_고장', '내비게이션_분실', '내비게이션_업데이트', '내비게이션_인터넷연결불가', '내비게이션_파손']
#inspect_type += ['단말기_배선', '단말기_신호불량', '단말기_탈거', '단말기_파손']
#inspect_type = ['브레이크_소음']
for inspect in inspect_type:
  total_case_trigger = []
  total_case_check = []
  total_case_figure = []
  total_case_action = []

  groupdf=df.groupby('lesion_detail_type').get_group(inspect) 
  lesion_id=groupdf['lesion_id']
  lesion_id=set(lesion_id.values.tolist())
  for id in lesion_id:
    groupdf2=groupdf.groupby('lesion_id').get_group(id).sort_values(by='memo_id') # 시간순으로 나열 
    groupdf2=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf3=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'ㅁ|ㄴ', repl='\n', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z:\s]', repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(문의[\s]?구분[\s]*[아래\s중\s택1]*[\\n]*[ㄴ]*)|(문의[\s]?내용[\s]*[ㅁ|ㄴ]*)', repl=r'문의내용: ', regex=True)  # '문의 구분 택'의 경우
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[\s]*:[xXo0O/(완료)(확인불가) ]*', repl=r':', regex=True) # 여부 ox표시
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=customer_info(), repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=service_info(), repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'부위[\s]*:', repl=r'부위는 ', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'긴출', repl=r'긴급출동', regex=True)

    content = groupdf2['memo'].values.tolist()
    origin_content = groupdf3['memo'].values.tolist()  # 원본
    print("============================================\n{}▶ Origin: ".format(inspect_type[inspect_num]), origin_content)

    # 첫부분 원소가- Trigger
    figure, check, trigger = track_Trigger(content[0]) 
    total_case_trigger.append(trigger)
    total_case_check.append(check)
    total_case_figure.append(figure)

    # 다음 원소에도 Trigger가 존재하는가 
    point = 0
    for i in range(1, len(content)):
        if factor_trgr.search(str(content[i])):
          figure, check, trigger = track_Trigger(content[i]) # 로직 분석기 돌리기
          total_case_trigger.append(trigger)
          total_case_check.append(check)
          total_case_figure.append(figure)
          point = i
    else: 
          #try:
            action = origin_content[point+1:]
            action = refine_action(action) # Action 중에서 필요한 정보만 추출하기
            print('차후수리: ', action)
            #total_case_action.append(action)
            total_case_action += action
          #except:
           #pass

  dic_trigger[inspect_num]= total_case_trigger
  dic_check[inspect_num]= total_case_check
  dic_figure[inspect_num]= total_case_figure
  dic_action[inspect_num]= total_case_action

  print('\n\n*****************************************************************************************************************')
  print(" >> ", inspect_type[inspect_num])
  print(total_case_trigger)
  print(total_case_check)
  print(total_case_figure)
  print(total_case_action)
  print('*****************************************************************************************************************\n\n')

  inspect_num+=1
  if inspect_num > 10:
    break


브레이크_소음▶ Origin:  ['[긴급출동 접수]\n - 접수내용:브레이크\n - 상세 사유: 브레이크에서 끼익소리가남\n - 연락 가능한 번호: 010 2464 7892\n - 일반도로 현주소: 서울시 구로구 도림로 72 라라동물병원 주차장\n - 현장 고객 유무: 0\n - 입고지:\n - 고객 과실 여부: X\n - 고객 과실 사유: \n - 관제 예약 URL : ', '1. 증상 상세내용 : \nㅁ문의내용\nㄴ브레이크에서 끼익소리가 난다고 함\nㅁ확인 내용\nㄴ본인 확인 0\n\nㅁ안내\nㄴ긴급출동 접수안내\nㄴ현주소 확인\nㄴ현장 납부비용 없으며 과실 여부에 따라 추후 청구될 수 있음\nㄴ긴급출동 기사가 현장에서 비용 납부 고객센터 측으로 문의 안내 필요\nㄴ긴급출동 기사는 차량 상태 및 비용 관련 문의에 대해서는 답변하지 않음\nㄴ지연시간 무상연장 요청\nㄴ약속내용\nㄴ없음', 'ㅁ문의내용\nㄴ브레이크에서 끼익소리가 난다고 함\nㅁ확인 내용\nㄴ본인 확인 0\n\nㅁ안내\nㄴ긴급출동 접수안내\nㄴ현주소 확인\nㄴ현장 납부비용 없으며 과실 여부에 따라 추후 청구될 수 있음\nㄴ긴급출동 기사가 현장에서 비용 납부 고객센터 측으로 문의 안내 필요\nㄴ긴급출동 기사는 차량 상태 및 비용 관련 문의에 대해서는 답변하지 않음\nㄴ지연시간 무상연장 요청\nㄴ약속내용\nㄴ없음', '원예약번호 : 35637059\n1. 증상 상세내용 : \n\t[긴급출동 접수]\n- 접수내용:브레이크\n- 상세 사유: 브레이크에서 끼익소리가남\n- 연락 가능한 번호: 010 24647892\n- 일반도로 현주소: 서울시 구로구 도림로 72 라라동물병원 주차장\n- 현장 고객 유무: 0\n- 입고지:\n- 고객 과실 여부: X\n- 고객 과실 사유:\n- 관제 예약 URL', '서울특별시 동작구 상도로 406 / 마인디즈 입고', '신규접수->처리중 (2021-01-15 13:39:32)', '뒤패드 뒤디스크 교환', '처리중->조치완료 (2021-01-18 15:21:05)']

In [208]:
divider = {'Trigger':'[으]*로$', 'Action':' 이후 | 후 | 후에', 'Action_fact':'교체|진행|조정|작업|청소'}

origin_action = []
action = []
trigger_sec = []
for i in dic_action[0]:
  
  if re.compile(divider['Action']).search(i):
    origin_action.append(i)
    if re.compile(divider['Trigger']).search(i):
      trigger_sec.append(re.split(divider['Trigger'], i)[0])
      action.append(re.split(divider['Action'], re.split(divider['Trigger'], i)[1])[0])
    else:
      action.append(re.split(divider['Action'], i)[0])
  elif re.compile(divider['Action_fact']).search(i):
    origin_action.append(i)
    i = re.sub(r'완료', '', i)
    #i = re.sub(divider['Action_fact'], divider['Action_fact']+'/', i)
    print("변경:", i)
    action.append(i)
  else:
    pass#print(i)
# for i in range(len(origin_action)):
#   print("결과:", origin_action[i],"->", action[i])

변경: 패드 디스크 교체
변경: 앞에  캘리퍼 교체
변경: 브레이크실린더 라이닝 슈 교체
변경: 드럼 청소 및 라이닝 가루 제거
변경: 타이어 2 본 교체 패드 교체 디스크 교체
변경: 전라 이닝 교체
변경: 드럼  교체
변경: 패드 디스크 교체
변경: 패드 교체 디스크 교체 타이어 2 본 교체
변경: 패드 교체
변경: 디스크 교체 패드 디스크 교체
변경: 패드 교체한지 얼마 안 돼서 소음 발생  타이어 2 본 얼라인먼트 진행
변경: 엔진 경고 등 점등되었으나 진단기로 확인 결과 문제없음으로 기억 소거 진행
변경: 패드 교체  타이어 교체
변경: OPT 테스트 진행
변경: 패드 교체
변경: 디스크 교체
변경: 패드 교체
변경: 디스크 교체
변경: OPT 테스트 진행
변경: 패드 디스크 교체 타이어  타이어 대미지로 교체
변경: 타이어 교체
변경: 휠얼라인먼트 조정
변경: 브레이크 오일 교체
변경: 패드 교체 패드 교체 디스크 교체 
변경: 패드 디스크 교체
변경: 패드가 교체하지 얼마 안 돼서 소음 나는 것 같다고 합니다
변경: 패드 교체
변경: 패드 교체 했습니다
변경: 패드 디스크 교체
변경: 패드 교체
변경: 뒤 분진가루 청소
변경: 교체  얼라인먼트 조정  패드 교체  브레이크디스크 교체 
변경: 패드 디스크 교체
변경: 브레이크 TS 진행
변경: 브레이크 작동 시 소음 크고 패드 완전 마모 상태입니다 입고 교체 필요합니다
변경: 패드  디스크 교체
변경: 오일 향균 와이퍼 교체
변경: 엔 와 향 교체 에어컨 필터 커버 장착 제 동시 소음 미발생
변경: 디스크 교체 패드 교체 타이어 2 본 교체 얼라인먼트 진행
변경: 패드 교체
변경: 패드 교체
변경: 패드 교체  팬 벨트 조정 디스크 연마 작업
변경: 교체 
변경: 시운전 결과 브레이크 소음 없음 전륜 타이어 2 본 교체
변경: 시운전 및 브레이크 소음 작업
변경: 전륜 타이어 2 본 교체 얼라인먼트
변경: 스패너 경고등 소거 엔진오일 교체 주기 안내 경고등
변경: 디스크 패드 교체
변경: 패드 패드

### 문장 유사도로 action부분 내용 유사 그룹끼리 묶기






In [209]:
#from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.add_dictionary('얼라인먼트|브레이크실린더', 'Noun')


# 문장 데이터 집어넣기
only_content = action
print(len(action))
origin_x_data = np.array(only_content)
origin_x_data_T = np.array(only_content)

for i, document in enumerate(origin_x_data):
      print('전:', document)
      nouns = twitter.morphs(origin_x_data[i]) # 형태소로 분리하기 (트위터_원본)
      origin_x_data_T[i] = ' '.join(nouns) 
      print('후:', origin_x_data_T[i])
      # nouns = morph_and_stopword(origin_x_data[i])
      # origin_x_data_T[i] = ' '.join(nouns) 
      # print('후:', origin_x_data_T[i])


origin_x_data_T = np.array(list(origin_x_data_T))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


62
전: 패드 디스크 교체
후: 패드 디스크 교체
전: 앞에  캘리퍼 교체
후: 앞 에 캘리 퍼 교체
전: 브레이크실린더 라이닝 슈 교체
후: 브레이크 실린더 라이닝 슈 교체
전: 드럼 청소 및 라이닝 가루 제거
후: 드럼 청소 및 라이닝 가루 제거
전: 타이어 2 본 교체 패드 교체 디스크 교체
후: 타이어 2 본 교체 패드 교체 디스크 교체
전: 패드 
후: 패드
전: 전륜 디스크 교체 패드는 샌딩 작업
후: 전륜 디스크 교체 패드 는 샌딩 작업
전: 전라 이닝 교체
후: 전라 이닝 교체
전: 드럼  교체
후: 드럼 교체
전: 패드 디스크 교체
후: 패드 디스크 교체
전: 패드 교체 디스크 교체 타이어 2 본 교체
후: 패드 교체 디스크 교체 타이어 2 본 교체
전: 패드 교체
후: 패드 교체
전: 디스크 교체 패드 디스크 교체
후: 디스크 교체 패드 디스크 교체
전: 패드 교체한지 얼마 안 돼서 소음 발생  타이어 2 본 얼라인먼트 진행
후: 패드 교체 한지 얼마 안 돼서 소음 발생 타이어 2 본 얼 라인 먼트 진행
전: 브레이크 소음 문제로 입고
후: 브레이크 소음 문제 로 입고
전: 엔진 경고 등 점등되었으나 진단기로 확인 결과 문제없음으로 기억 소거 진행
후: 엔진 경고 등 점등 되었으나 진 단기 로 확인 결과 문제 없 음으로 기억 소
전: 패드 디스크 교체
후: 패드 디스크 교체
전: 패드 교체  타이어 교체
후: 패드 교체 타이어 교체
전: OPT 테스트 진행
후: OPT 테스트 진행
전: 패드 교체
후: 패드 교체
전: 디스크 교체
후: 디스크 교체
전: 패드 교체
후: 패드 교체
전: 디스크 교체
후: 디스크 교체
전: OPT 테스트 진행
후: OPT 테스트 진행
전: 패드 디스크 교체 타이어  타이어 대미지로 교체
후: 패드 디스크 교체 타이어 타이어 대미지 로 교체
전: 타이어 교체
후: 타이어 교체
전: 휠얼라인먼트 조정
후: 휠얼 라인 먼트 조정
전: 브레이크 오일 교체
후: 브레이크 오일 교체
전: 패드 교체 패드 교체 디스크 교체

In [210]:
def make_correct_sentence(list_sentence, keyword_list):
  min_text = list_sentence[0]
  for i in list_sentence:
    for key in keyword_list:
      if key not in i:
        break
    else:
      if len(i) < len(min_text):
        min_text = i
  print('best:', min_text)
  return min_text

In [211]:

correct_sentence = [] # 추출된 correct sentence

def check_simillity(sentence_list):
  #print(sentence_list) # 담긴 문장 확인하기 
  vect = TfidfVectorizer()
  x_data = vect.fit_transform(sentence_list)
  sentence_list = sentence_list.tolist()
  cosine_similarity_matrix = (x_data * x_data.T)
  print('size: ', cosine_similarity_matrix.shape) 

  index_list = [] # 지워야할 정보
  cnt = 0
  for i in range(len(sentence_list)):
    if i not in index_list: # 이미 어떤 문장의 유사도에 포함되어있으면 검사하지말기
      cnt += 1
      sentences_for_keyword = [] # 유사한 문장담기
      vals = cosine_similarity(x_data[i], x_data) # 타깃하는 문장의 유사도 데이터 
      idx = vals.argsort()[0] # 오름차순에 해당하는 인덱스 정보를 담음 
      flat = vals.flatten() # 유사도 오름차순 정렬
      flat.sort()
      for index in range(len(flat)-1, -1, -1):
        if flat[index] < 0.9: # 유사한 정도
          break
        else:
            real_index = sentence_list.index(sentence_list[idx[index]]) # 실제 문장 리스트에서 몇번째 인덱스인지 flat은 유사도 오름차순된거라 실제 문장의 인덱스가 아님
            index_list.append(real_index) 
            sentences_for_keyword.append(sentence_list[real_index])
            
      if len(sentences_for_keyword) > 2: # 유사한 문장 그룹내의 키워드 추출     
        print('------------------------\n')
        for i in sentences_for_keyword:
          print(i)
        print(' \n▶ 「KRWordRank」형태소 분석:')
        try:
          okt_keyword = keyword_tracker_KR(sentences_for_keyword)
          print(okt_keyword)
        except: pass
        if len(okt_keyword) > 1: # 키워드가 하나이상이고
          main_sentence  = make_correct_sentence(sentences_for_keyword, okt_keyword)
          correct_sentence.append(main_sentence)
      else:
        correct_sentence.append(sentences_for_keyword[0])
  print(cnt)
  for i in set(correct_sentence):
    print(i)
  return 
check_simillity(origin_x_data)
print('\n\n*****************************************************************************************************************')
check_simillity(origin_x_data_T)

size:  (62, 62)
------------------------

패드 디스크 교체
패드 디스크 교체
패드 디스크 교체
패드 디스크 교체
패드  디스크 교체
패드 디스크 교체
패드 디스크 교체
디스크 패드 교체
디스크 패드 교체
디스크 교체 패드 디스크 교체
패드 교체 패드 교체 디스크 교체 
 
▶ 「KRWordRank」형태소 분석:
scan vocabs ... 
num vocabs = 11
done
      교체:	1.4786
      패드:	1.2128
     디스크:	0.9054
['교체', '패드', '디스크']
best: 패드 디스크 교체
------------------------

패드 디스크 교체
패드 디스크 교체
패드 디스크 교체
패드 디스크 교체
패드  디스크 교체
패드 디스크 교체
패드 디스크 교체
디스크 패드 교체
디스크 패드 교체
디스크 교체 패드 디스크 교체
패드 교체 패드 교체 디스크 교체 
 
▶ 「KRWordRank」형태소 분석:
scan vocabs ... 
num vocabs = 11
done
      교체:	1.4786
      패드:	1.2128
     디스크:	0.9054
['교체', '패드', '디스크']
best: 패드 디스크 교체
------------------------

패드 교체
패드 교체
패드 교체
패드 교체
패드 교체
패드 교체
패드 교체
패드 교체
패드 교체
패드 교체
패드 패드 교체
 
▶ 「KRWordRank」형태소 분석:
scan vocabs ... 
num vocabs = 6
done
      패드:	1.0492
      교체:	0.9508
['패드', '교체']
best: 패드 교체
------------------------

패드 디스크 교체
패드 디스크 교체
패드 디스크 교체
패드 디스크 교체
패드  디스크 교체
패드 디스크 교체
패드 디스크 교체
디스크 패드 교체
디스크 패드 교체
디스크 교체 패드 디스크 교체
패드 교체 패드 교체 디스크 교체 
 
▶ 「KRWo

In [ ]:
for i in correct_sentence:
  print(i)
print(len(correct_sentence))

In [17]:
# 원본 데이터 보관
import copy
temp_dic_trigger = copy.deepcopy(dic_trigger)
temp_dic_figure = copy.deepcopy(dic_figure)
temp_dic_check = copy.deepcopy(dic_check)
temp_dic_action = copy.deepcopy(dic_action)

In [18]:

#factor_emergency = re.compile(r'긴급[\s]*출동[\s]*[접수]*|하이패스') 

def refine_figure(figure_list):
    money_figure = '금액|요금|결제|무상|환불|청구|보상|쿠폰|차액|상급자|납부|할인|대여료|비용|유효[\s]*기간|수긍|연장|과실|패널티|페널티|[쏘카|소카|동일|원]존|대차|협의|동일|송부|문자|링크|도착|취소|변경|죄송|불편드|양해|사과|조치|전화|점검|담당자|알겠|주장|일단|그냥|'
    money_figure += '통화[\s]*종료|감사|분실물|반납|그냥[\s]*[이용|운행|반납]|첨부|위치|추가료|이동|주차|주행[\s]*중|운행[\s]*중|어렵|연락|거부|사진|항의|접수|기사|시간|통화|추후|택시비|영수증|완료|요청|기록|부서|불편|종료|불만|상담[원|사]|안내|이름|이용|소통|고객|안된다|안보'
    money_figure = re.compile(money_figure) 
    #문[\s]*(제어|잠금|열|닫)
    factor_figure = '안내[\s]*[약속[\s]*내용|처리[\s]*사항|처리[\s]*내용|사항]|안내사항|tcs|TCS|티씨에스|[네|내]비게이션|[네|내]비|음성|재부팅|오토|충전소|원격| '
    factor_figure += '누유|시거잭|배출가스|가리개|글로브박스|업데이트|내장제|공기압배선|후방| 센서|ESC |탈거|환경이슈|전좌|버튼|전우|발판매트|만차|인터넷|목받침대|블랙[\s]*박스|시트조절|하부|안심번호|관리인요청|파손|얼라이먼트|신호불량|타는냄새|위치조절|AUX|출차불가|배터리|휠|'
    factor_figure += '블루투스|스패너|혼유|서비스알림|펑크|상부|엑셀|급발진|고장|[선|썬]루프|썬|[선|썬]바이저|시동|USB|포트|히터|타이어|꺼|들리|얼어|언|따뜻|차가운|찬|깜[빡|박]|알림|쏠|찢|뽑|꼽|닫|충돌|방지|보조|에이필러|바람[\s]*빠'
    factor_figure += '태깅|연결|외부스티커|에어백|RPM|rpm|계기판|요소수|위생불량|DPF|gps|GPS|주유|스마트[\s]*키|범퍼|[sdSD]+[\s]*카드|경보|소음|마스터키|소리|악취|구토|ABS|고장|손상|차량[\s]*떨림|떨리|작동|ADAS|번호판|오류|유리|오일|외관|부족|카메라|냄새|담뱃재|온도게이지|변속기|이상|보관함|코드절상|'
    factor_figure += '신호|탐색|기름|방전|충전|연료|견인|인식|미러|사이드|손잡이|글로브[\s]*박스|바퀴|마모|공업사|하이패스|불[\s]*빛|라이트|캡|분실|볼륨|고정|네트[\s]*워크|공조|장치|유리|습기|수동|제어|잠금|열|닫|전조등|실내등|경고등|주행등|제동등|점등|경고|후미등|시야[\s]*[확보]*|조수석|[자|좌]석|재시작|[하|상]향|각도|컨트롤[\s]*패드|스크[레|래]치|단차|내부|엔진|운전석|보험|동파|모드|방향|판단|편의|품목|접촉| 훼손|통신|초기화|전원|안나|내비|사고조사|실행|화면|와이파이|창문|증상|후진|기어|오디오|트렁크|단말기|장착물|핸들떨림|후좌|열선|후측방 충돌|밀림|해제불가|본넷|후우|차세제어|에어컨|중계기점검|브레이크|배터리방전|와이퍼|옥스트라|변속레버|외부위생불량 |카드오류|도어|체크|연동|비용발생|파스|교체필요|입차불가|전구|커넥터|안전벨트|키아웃|핸들|열선시트|비치함|거치대|담배|시트|위생|[와|워]셔액| 차량위치확인불가|썬팅|경적|키인식|꺼짐|동물털|충전[\s]*카드|주유[\s]*카드'
    factor_figure = re.compile(factor_figure) 
    remove = [] # 지운 목록 담기

    for i in figure_list[:]:
        m = money_figure.search(i)
        f = factor_figure.search(i)
        if not f: 
          #print(i)
          remove.append(i)
          figure_list.remove(i)
        else:
          pass
          # print(re.findall(factor_figure, i), ": ")
          # if '긴급' in i: emergency.append(i); case.remove(i)
          # else: print(i)
    #print("지운거", remove)
    return figure_list

for inspect_num, total_case_figure in temp_dic_figure.items():
    for case in total_case_figure[:]:
        if len(case) == 0:
          total_case_figure.remove(case)
          continue
        print(refine_figure(case))

for inspect_num, total_case_figure in temp_dic_figure.items():
    print(inspect_type[inspect_num]," ({}개): ".format(len(total_case_figure)), total_case_figure)

'운행[\s]' : 운행가능여부를 상담사가 파악한게 중요한 요소인가
28 차량정차후 비상등 켜논상태로 사진촬영후 보내달라안내  비상등이 figure에 속하는지
영수증 첨부 내용 - 카드안돼서 발생하는 경우 워셔액 산 경우 
경찰관련 압수수색영장 관련해서도 처리해야하는가

'기사' : 견인 기사를 보내거나 긴급출동한 판단도 figure인지

혹시 figure에 사용할만한 단어리스트 줄수있는지    
비접촉 주장   
보험    
차케어, 소닉붐,  모본 점검 완료'   
주유카드, 하이패스의 경우와 같이  action이 환불과 같은 비용적인 부분인 경우는?


In [ ]:
target_figure = re.compile(r'긴급[\s]*출동') #,, 종료 ,, 하심, 동일   담당자, 없 이대로
long_len = []
factor_figure = '안내[\s]*[약속[\s]*내용|처리[\s]*사항|처리[\s]*내용|사항]|안내사항|tcs|TCS|티씨에스|[네|내]비게이션|[네|내]비|음성|재부팅|오토|충전소|원격| '
factor_figure += '누유|시거잭|배출가스|가리개|글로브박스|업데이트|내장제|공기압배선|후방| 센서|ESC |탈거|환경이슈|전좌|버튼|전우|발판매트|만차|인터넷|목받침대|블랙[\s]*박스|시트조절|하부|안심번호|관리인요청|파손|얼라이먼트|신호불량|타는냄새|위치조절|AUX|출차불가|배터리|휠|'
factor_figure += '블루투스|스패너|혼유|서비스알림|펑크|상부|엑셀|급발진|고장|[선|썬]루프|썬|[선|썬]바이저|시동|USB|포트|히터|타이어|꺼|들리|얼어|언|따뜻|차가운|찬|깜[빡|박]|알림|쏠|찢|뽑|꼽|닫|충돌|방지|보조|에이필러|바람[\s]*빠'
factor_figure += '태깅|연결|외부스티커|에어백|RPM|rpm|계기판|요소수|위생불량|DPF|gps|GPS|스마트[\s]*키|범퍼|[sdSD]+[\s]*카드|경보|소음|마스터키|소리|악취|구토|ABS|고장|손상|차량[\s]*떨림|떨리|작동|ADAS|번호판|오류|유리|오일|외관|부족|카메라|냄새|담뱃재|온도게이지|변속기|이상|보관함|코드절상|'
factor_figure += '신호|탐색|기름|방전|충전|연료|견인|인식|미러|사이드|손잡이|글로브[\s]*박스|바퀴|마모|공업사|하이패스|불[\s]*빛|라이트|캡|분실|볼륨|고정|네트[\s]*워크|공조|장치|유리|습기|수동|제어|잠금|열|닫|전조등|실내등|경고등|주행등|제동등|점등|경고|후미등|시야[\s]*[확보]*|조수석|[자|좌]석|재시작|[하|상]향|각도|컨트롤[\s]*패드|스크[레|래]치|단차|내부|엔진|운전석|보험|동파|모드|방향|판단|편의|품목|접촉|훼손|통신|초기화|전원|안나|내비|사고조사|실행|화면|와이파이|창문|증상|후진|기어|오디오|트렁크|단말기|장착물|핸들떨림|후좌|열선|후측방 충돌|밀림|해제불가|본넷|후우|차세제어|에어컨|중계기점검|브레이크|배터리방전|와이퍼|옥스트라|변속레버|외부위생불량 |카드오류|도어|체크|연동|파스|교체필요|입차불가|전구|커넥터|안전벨트|키아웃|핸들|열선시트|비치함|거치대|담배|시트|위생|[와|워]셔액| 차량위치확인불가|썬팅|경적|키인식|꺼짐|동물털|충전[\s]*카드|주유[\s]*카드'
factor_figure = re.compile(factor_figure) 

money_figure = re.compile(r'[패|페]널티|영수증|주유소')


for inspect_num, total_case_figure in temp_dic_figure.items():
    for case in total_case_figure:
          if len(case) == 0:
            total_case_figure.remove(case)
            continue
          for i in case:
              t = target_figure.search(i)
              m = money_figure.search(i)
              f = factor_figure.search(i)
              if t:
                if f: 
                  print(i)
                elif m:
                  #print(re.findall(factor_figure, i), ":", i)
                  long_len.append(i)
                else:
                  print(i)
print('-------------------------------------------------------------')
long_len.sort(key=lambda x:len(x))
for i in long_len:
    print(len(i), i)
# for inspect_num, total_case_figure in dic_figure.items():
#     print(inspect_type[inspect_num],": ({}개)".format(len(total_case_figure)), total_case_figure)

# 접수 분리 내용 

In [ ]:
# csv 파일로 저장하기 


f = open('/content/drive/MyDrive/Trigger_list_31_46.txt','w')
f_C = open('/content/drive/MyDrive/Trigger_list_COUNT.txt','w')

cnt =0
# (사건 타입 번호 - 문의 사항) 저장
for idx, trigger in dic_trigger.items():
  trigger = set(trigger) # 1차 중복 제거
  
  # \n으로 나누기 전 문장 내에 \n으로 끊길 가능성 있는거 이어주기
  trigger_str= '\n'.join(trigger)
  trigger_str=re.sub(r'(\n+)', '\n', trigger_str)
  trigger_str=re.sub(r'\nㄴ|ㄴ', ' ', trigger_str)
  trigger_str=re.sub(r'(손상고장[\s][부위는]*|파손고장[\s][부위는]*|문의내용|문의사항|증상[\s]*상세내용|접수[\s]*상세[\s]*사유|접수내용|조치사항|확인사항|확인내용|현장조치)[:\s]*', '', trigger_str)
  trigger_str=re.sub(r'(여부[oOxX:]*)', '', trigger_str)
  trigger_str=re.sub(' +', ' ', trigger_str) # 다중 공백 제거
  trigger_str=re.sub(r'( )*\n( )*', '\n', trigger_str)
  trigger_str=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\\n ]|(긴급출동 접수)\n', '', trigger_str) 
  trigger_str=re.sub('\n', '-', trigger_str) 

  #keyword_tracker2(trigger_str) # 형태소 추출기로 하면 정확도가 낮음 
  #trigger_str=correction(trigger_str)
  #print(trigger_str)
  # 맞춤법 검사기 돌리기 위해 특수문자 제거
  b = list(set(trigger_str.split('-')))

  print(inspect_type[idx],  '({}개): '.format(len(b)), sorted(b))
  b.sort(key=lambda x:len(x))
  f_C.write('\n{} ({}개): '.format(inspect_type[idx],  len(b)))
  f_C.write(str(sorted(b)))

  
  if len(b) > 100:
    for B in b:
      if len(B) > 6:
        #id\tdocument\tlabel\n : 형식
        # wr.writerow([idx, B]) # cs
        B=correction(B)
        f.write(str(idx)+'\t'+B+'\t'+str(cnt)+'\n') # 각 리스트에 담긴 문장을 쓴다.
    cnt += 1
  if cnt == 3: break  


# 모델학습

### 라이브러리 설치 및 불러오기

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
print(vocab)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

### 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
#dataset = pd.read_table('/content/drive/MyDrive/Trigger_list.txt',sep='\t',header=None,names=['id','문의사항', 'Trigger'])
dataset = pd.read_table('/content/drive/MyDrive/Trigger_list_31_46.txt',sep='\t',header=None,names=['id','문의사항', 'Trigger'])

dataset.head()

In [ ]:
# # 문자를 수치화하기 
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(dataset['문의사항'])
# dataset['문의사항'] = encoder.transform(dataset['문의사항'])
# dataset.head()

In [ ]:
# 가져온 내 데이터를 훈련 테스트셋과 테스트 데이터 셋으로 나누기 
from sklearn.model_selection import train_test_split   
#dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=42)
#print("train shape is:", len(dataset_train))
#print("test shape is:", len(dataset_test))
# 튜토리얼에서 사용하는 데이터 로드 방식
# dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Trigger_list.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Trigger_list.txt", field_indices=[1,2], num_discard_samples=1)
dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Trigger_list_31_46.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Trigger_list_31_46.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
print(type(dataset_train))
for i in range(len(dataset_train)):
  print(dataset_train[i])

### 모델학습

In [ ]:
## Setting parameters
max_len = 164  # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32 # 본 64
warmup_ratio = 0.1
num_epochs = 180
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=cnt+1,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


 

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
 
    # train 데이터 학습
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # test 데이터 학습
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
unseen_test = pd.DataFrame([['타이어 파스', 7]], columns = [['질문 내용', '질병명']])
unseen_values = unseen_test.values
 
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)
 
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)